In [1]:
import os 
from crewai import Agent, Task, Crew, Process
from datetime import datetime, timedelta
from crewai_tools import CSVSearchTool

In [2]:
from dotenv import load_dotenv
_ = load_dotenv() 
API_KEY = os.getenv("OPENAI_API_KEY")


In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=API_KEY)

In [4]:
import json
with open("mock_catalog.json", "r", encoding="utf-8") as f:
    catalog_data = json.load(f)

### Agente 1 - Orquestrador

In [5]:
orquestrador = Agent(
    role="Orquestrador Editorial",
    goal=(""" 
        1 - Detectar a intenção do usuário entre: DETALHES, ONDE_COMPRAR ou SUPORTE.
        2 - Coletar somente os dados mínimos faltantes:
           - DETALHES: title (obrigatório).
           - ONDE_COMPRAR: title (obrigatório) e city (opcional). Se city não vier, delegar assim mesmo;
        o agente de Catálogo aplicará fallback em availability['Online'].
           - SUPORTE: name, email, subject, message (todos obrigatórios).
        Delegar ao agente responsável (Catálogo/Comercial ou Suporte) e aguardar o retorno. Consolidar uma resposta curta
        e objetiva para o usuário, mantendo o contexto da sessão. Registrar logs estruturados (intent, params, agente_destino, duração/erros).
    """),
    backstory=(""" 
        Primeiro ponto de contato do chat. Atua como roteador inteligente: 
        faz perguntas só quando necessário, não chama ferramentas diretamente, 
        e garante que cada solicitação seja atendida pelo agente certo.
    """),
    verbose=True,          
    memory=True,           
    allow_delegation=True, 
    max_iterations=3,                  
)


### Agente 2 - Analista Editorial de Livros

In [6]:
analista_catalogo_comercial = Agent(
    role="Agente de Catálogo/Comercial",
    goal=""" 
        Atender intenções DETALHES e ONDE_COMPRAR consultando o catálogo via ferramentas. 
        DETALHES: requer 'title'; retornar somente os campos existentes: 
        title, author, imprint, release_date (DD/MM/AAAA), synopsis. 
        ONDE_COMPRAR: requer 'title' e city (opcional); se city ausente ou indisponível, 
        retornar availability['Online'] como fallback. 
        Se o livro não for encontrado, informar educadamente que não está no catálogo.
    """,
    backstory="""
        Especialista no acervo da editora. Usa apenas as ferramentas oficiais para evitar erros
        e responde de forma curta e direta.
    """,
    verbose=True,
    memory=True,
    allow_delegation=False,
    max_iterations=3,
)


In [7]:
get_book_details_task = Task(
    description="""
        Você vai receber um título de livro em {title}. Consulte o catálogo usando 
        suas ferramentas oficiais e traga só o que realmente existe no dado. 
        Se encontrar, responda de forma curta e direta. Se não encontrar, diga 
        claramente que não está no catálogo.
    """,
    expected_output=""" 
        Retorne APENAS um JSON enxuto (sem comentários), com estes campos quando disponíveis:
        {
          "title"
          "author"
          "imprint"
          "release_date"
          "synopsis"
        }
    Se não encontrar o livro, retorne: {"not_found": true, "title": "{title}"}
    
    Esse são os dados que nós temos dos livros:
    {
  "books": [
    {
      "title": "A Abelha",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Elo Editora",
      "release_date": "15/04/2022",
      "synopsis": "Uma obra delicada que explora o universo das abelhas e sua importância para a natureza. Com ilustrações de Olavo Costa, o livro é uma jornada poética e educativa.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Cultura"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A Baleia-azul",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "10/09/2023",
      "synopsis": "Acompanhe a vida majestosa do maior animal do planeta. Com ilustrações de Felipe Tognoli, esta obra mergulha nos mistérios dos oceanos de forma cativante.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br", "Magazine Luiza", "Submarino"]
      }
    },
    {
      "title": "A Bicicleta do Tempo",
      "author": "Daniela Pinotti e Marcelo Maluf",
      "imprint": "Elo Editora",
      "release_date": "20/05/2021",
      "synopsis": "Uma viagem fantástica sobre duas rodas que atravessa gerações e histórias. Ilustrado por Matheus Furtado, este livro fala sobre memória, amizade e as voltas que a vida dá.",
      "availability": {
        "Curitiba": ["Livrarias Curitiba"],
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Americanas.com"]
      }
    },
    {
      "title": "A Borboleta",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "12/02/2024",
      "synopsis": "Celebra a beleza e a transformação da vida através do ciclo da borboleta. As ilustrações de Rosana Ferreira dão cor e vida a esta delicada narrativa.",
      "availability": {
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A Cinderela dos Cajueiros",
      "author": "Severino Rodrigues",
      "imprint": "Elo Editora",
      "release_date": "30/07/2020",
      "synopsis": "Uma releitura do clássico conto de fadas, ambientado no sertão nordestino. Com a arte de Rodrigo Mafra, a história ganha novos sabores e cores.",
      "availability": {
        "Salvador": ["Livraria LDM"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A conquista de um sonho",
      "author": "Regina Drummond",
      "imprint": "Perabook",
      "release_date": "18/11/2022",
      "synopsis": "Uma história inspiradora sobre perseverança e a importância de lutar por seus objetivos. Ilustrado pela própria autora, Regina Drummond.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Leitura"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Cuca",
      "author": "Theo de Oliveira",
      "imprint": "Elo Editora",
      "release_date": "01/08/2021",
      "synopsis": "Mergulhe no folclore brasileiro com esta história sobre a Cuca, a temível feiticeira com cabeça de jacaré. Com vibrantes ilustrações de Danilo Taqueto.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Salvador": ["Livraria LDM"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "A Escola Lá Fora",
      "author": "Janaina Tokitaka",
      "imprint": "Perabook",
      "release_date": "25/03/2023",
      "synopsis": "Uma reflexão sobre o aprendizado que acontece fora das paredes da sala de aula. Escrito e ilustrado por Janaina Tokitaka, o livro convida a explorar o mundo.",
      "availability": {
        "São Paulo": ["Livraria Cultura", "Blooks Livraria"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A estranha atração dos planetas vizinhos",
      "author": "Claudio Fragata",
      "imprint": "Elo Editora",
      "release_date": "09/06/2024",
      "synopsis": "Uma história sobre conexões inesperadas e as forças que nos unem. Escrito e ilustrado com a sensibilidade de Claudio Fragata.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br (Pré-venda)"]
      }
    },
    {
      "title": "A Fabulosa Arca de Noá",
      "author": "Leandro Carloni",
      "imprint": "Perabook",
      "release_date": "14/10/2021",
      "synopsis": "Uma divertida e poética releitura da história da Arca de Noé, com personagens cativantes e ilustrações encantadoras de Fernanda Ozilak.",
      "availability": {
        "Curitiba": ["Livrarias Curitiba"],
        "Porto Alegre": ["Livraria Cameron"],
        "Online": ["Amazon.com.br", "Americanas.com"]
      }
    },
    {
      "title": "A Família Formiga",
      "author": "Augusto Pessoa",
      "imprint": "Elo Editora",
      "release_date": "22/08/2022",
      "synopsis": "Descubra o dia a dia de uma agitada família de formigas e aprenda sobre trabalho em equipe e união. Com as divertidas ilustrações de Heitor Neto.",
      "availability": {
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Fantástica Viagem de Okiké, o Macaquinho de Nariz Vermelho",
      "author": "G. Guillen e Marcel Tenório da",
      "imprint": "Perabook",
      "release_date": "03/03/2020",
      "synopsis": "A jornada de um macaquinho especial em busca de aceitação e de seu lugar no mundo. As ilustrações de Cris Mendes dão vida a essa aventura na floresta.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A Formiga",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Elo Editora",
      "release_date": "19/04/2022",
      "synopsis": "Um olhar poético sobre a vida da formiga, sua força e persistência. Com a arte delicada de Ana Laura Alvarenga, o livro encanta pela simplicidade.",
      "availability": {
        "Online": ["Loja Elo Editorial", "Amazon.com.br"]
      }
    },
    {
      "title": "A Girafa",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "11/07/2023",
      "synopsis": "Explore a savana e conheça mais sobre a elegante girafa. Um livro informativo e visualmente rico, com ilustrações de Tainan Rocha.",
      "availability": {
        "São Paulo": ["Livraria Cultura"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A girafa que comeu a lua",
      "author": "Francine Bittencourt",
      "imprint": "Elo Editora",
      "release_date": "08/12/2021",
      "synopsis": "Uma história lúdica sobre sonhos, imaginação e até onde a curiosidade pode nos levar. Com as ilustrações mágicas de Vanessa Prezoto.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "A História de Virgulino",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Perabook",
      "release_date": "21/06/2022",
      "synopsis": "Uma narrativa inspirada na figura de Lampião, adaptada para o público jovem. Com a arte em estilo de xilogravura de Adelino Francisco dos Santos Neto.",
      "availability": {
        "Salvador": ["Livraria LDM"],
        "Online": ["Estante Virtual", "Amazon.com.br"]
      }
    },
    {
      "title": "A ilha do tesouro",
      "author": "Robert Louis Stevenson",
      "imprint": "Elo Editora",
      "release_date": "15/01/2022",
      "synopsis": "A clássica aventura de piratas e mapas, em uma edição especial com a arte moderna de Weberson Santiago, que dá nova vida à jornada de Jim Hawkins.",
      "availability": {
        "São Paulo": ["Livraria Cultura", "Livraria Leitura"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Porto Alegre": ["Livraria Cameron"],
        "Online": ["Amazon.com.br", "Americanas.com", "Estante Virtual"]
      }
    },
    {
      "title": "A jornada de Roweõ",
      "author": "Angela Pappiani",
      "imprint": "Perabook",
      "release_date": "29/09/2020",
      "synopsis": "Baseado em uma história real, o livro narra a jornada de um jovem Xavante. Com as impressionantes ilustrações de Mauricio Negro, é uma obra sobre cultura e resistência.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A Lenda da Vitória-régia",
      "author": "Alice Bella",
      "imprint": "Elo Editora",
      "release_date": "16/03/2021",
      "synopsis": "O reconto de uma das mais belas lendas do folclore amazônico. A arte de Danilo Taqueto captura a magia e o mistério da história da índia Naiá.",
      "availability": {
        "Manaus": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Lenda do Guaraná",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "16/03/2021",
      "synopsis": "Descubra a origem mítica do guaraná nesta adaptação de uma famosa lenda indígena. As ilustrações de Danilo Taqueto complementam a narrativa.",
      "availability": {
        "Manaus": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "A mágica história de um livro encontrado",
      "author": "Stella Maris Rezende",
      "imprint": "Elo Editora",
      "release_date": "11/11/2022",
      "synopsis": "Um livro que fala sobre o amor pelos livros e as aventuras que eles podem nos proporcionar. Com ilustrações de Sid Meireles, é uma ode à leitura.",
      "availability": {
        "Belo Horizonte": ["Livraria Leitura"],
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A megera domada",
      "author": "William Shakespeare",
      "imprint": "Perabook",
      "release_date": "25/02/2023",
      "synopsis": "Uma adaptação moderna e acessível da clássica comédia de Shakespeare, sobre os embates entre Catarina e Petrúquio. Com a arte arrojada de Rodrigo Mafra.",
      "availability": {
        "Online": ["Amazon.com.br", "Loja Elo Editorial", "Americanas.com"]
      }
    },
    {
      "title": "A menina da varanda",
      "author": "Leo Cunha",
      "imprint": "Elo Editora",
      "release_date": "10/03/2018",
      "synopsis": "Uma garota curiosa observa o mundo da sua varanda, criando histórias sobre as pessoas que passam. Uma obra sensível ilustrada por Rogério Coelho.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Leitura"],
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial", "Magazine Luiza"]
      }
    },
    {
      "title": "A Minhoca",
      "author": "Paulla Thompson",
      "imprint": "Perabook",
      "release_date": "07/07/2024",
      "synopsis": "De forma divertida, este livro mostra a importância das minhocas para o solo e para o meio ambiente. Com as bem-humoradas ilustrações de Monge.",
      "availability": {
        "Online": ["Amazon.com.br (Pré-venda)"]
      }
    },
    {
      "title": "A Minhoca com Dor de Barriga",
      "author": "Adriana Yazbek",
      "imprint": "Elo Editora",
      "release_date": "14/01/2020",
      "synopsis": "O que será que uma minhoca comeu para ficar com dor de barriga? Uma história divertida e cheia de imaginação, ilustrada por Isabella Guizalberti.",
      "availability": {
        "São Paulo": ["Livraria Cultura"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A Mula sem Cabeça",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Perabook",
      "release_date": "28/10/2021",
      "synopsis": "Mais uma lenda do folclore brasileiro ganha vida nesta obra. Descubra a história da Mula sem Cabeça com as ilustrações de Heitor Neto.",
      "availability": {
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A Poça d’Água",
      "author": "Ana Rapha Nunes",
      "imprint": "Elo Editora",
      "release_date": "18/05/2023",
      "synopsis": "Uma simples poça d'água pode ser um universo de descobertas e brincadeiras. Com as criativas ilustrações de Paula Kranz.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Curitiba": ["Livrarias Curitiba"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A poesia do nome",
      "author": "Vários Autores",
      "imprint": "Perabook",
      "release_date": "05/11/2024",
      "synopsis": "Uma antologia que celebra o poder e a beleza dos nomes através de poemas e contos de diversos autores talentosos. Uma coletânea essencial.",
      "availability": {
        "Online": ["Loja Elo Editorial", "Amazon.com.br"]
      }
    },
    {
      "title": "A Princesa e o Jasmim",
      "author": "Georgina Martins",
      "imprint": "Elo Editora",
      "release_date": "12/04/2022",
      "synopsis": "Uma releitura poética de contos orientais, sobre uma princesa e uma flor de jasmim mágica. A arte de Ana Laura Alvarenga transporta o leitor para outro mundo.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Pulga",
      "author": "Alice Bella",
      "imprint": "Perabook",
      "release_date": "23/08/2023",
      "synopsis": "A vida de uma pulga aventureira que salta de um animal para outro em busca de emoção. Ilustrado de forma cômica por Rosana Ferreira.",
      "availability": {
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A Tartaruga",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Elo Editora",
      "release_date": "15/04/2022",
      "synopsis": "Uma história sobre paciência, sabedoria e o ritmo da natureza, contada pela perspectiva de uma tartaruga. Com as ilustrações de Olavo Costa.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A vaca que não parava de sonhar",
      "author": "Zil de Paula",
      "imprint": "Perabook",
      "release_date": "09/02/2021",
      "synopsis": "Conheça uma vaca que sonhava em voar, nadar e viajar pelo mundo. Uma história sobre imaginação e não ter medo de ser diferente, com ilustrações de Paula Kranz.",
      "availability": {
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Americanas.com"]
      }
    },
    {
      "title": "A viagem",
      "author": "Jean Claude Alphen",
      "imprint": "Elo Editora",
      "release_date": "27/06/2023",
      "synopsis": "Um livro de imagens sem texto que narra a emocionante viagem de um menino e seu barco. Escrito e ilustrado por Jean-Claude Alphen.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Cultura"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "Abecê do macunaíma: O herói da nossa gente",
      "author": "Claudio Fragata",
      "imprint": "Perabook",
      "release_date": "01/02/2024",
      "synopsis": "Uma introdução divertida e acessível ao universo de Macunaíma, o herói sem nenhum caráter de Mário de Andrade. Com a arte icônica de Ciça Fittipaldi.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "Além do mar",
      "author": "Ana Rapha Nunes",
      "imprint": "Perabook",
      "release_date": "20/10/2025",
      "synopsis": "Uma delicada história sobre conexão e saudade. Ilustrado por Rafa Antón, é um livro sobre as viagens que fazemos por dentro e por fora.",
      "availability": {
        "Online": ["Amazon.com.br (Pré-venda)", "Loja Elo Editorial (Pré-venda)"]
      }
    },
    {
      "title": "Ana bola e outras histórias corajosas",
      "author": "Eliana Martins",
      "imprint": "Elo Editora",
      "release_date": "13/05/2021",
      "synopsis": "Uma coletânea de contos sobre meninas fortes e corajosas que enfrentam seus medos. Com as expressivas ilustrações de Tainan Rocha.",
      "availability": {
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "Ao sol do novo mundo",
      "author": "Edy Lima",
      "imprint": "Perabook",
      "release_date": "10/08/2022",
      "synopsis": "A saga de uma família de imigrantes que chega ao Brasil em busca de uma nova vida. Uma narrativa histórica com a arte de Taísa Borges.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "As ceroulas do rei",
      "author": "Zil de Paula",
      "imprint": "Elo Editora",
      "release_date": "04/04/2023",
      "synopsis": "Uma releitura bem-humorada do conto 'A Roupa Nova do Rei', mostrando o que acontece quando a verdade é dita. Ilustrado por Marcelo Cardinal.",
      "availability": {
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "As Cores da Gatinha Mimi",
      "author": "Eliana Martins",
      "imprint": "Perabook",
      "release_date": "24/01/2022",
      "synopsis": "A gatinha Mimi muda de cor de acordo com suas emoções. Um livro encantador sobre sentimentos para os pequenos, com a arte de Cris Eich.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "As estranhas criaturas do Sr. K",
      "author": "Franz Kafka",
      "imprint": "Elo Editora",
      "release_date": "30/03/2024",
      "synopsis": "Uma adaptação de contos de Kafka para o público jovem, explorando o bizarro e o absurdo. Com a interpretação visual única de Nat Grego.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "As peripécias da senhora Adélia",
      "author": "Maria Amália Camargo",
      "imprint": "Perabook",
      "release_date": "17/10/2023",
      "synopsis": "Acompanhe as divertidas e inesperadas aventuras de uma senhora cheia de vida e energia. Ilustrado por Vanessa Prezoto.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "As Torres",
      "author": "Orlando Pedroso",
      "imprint": "Elo Editora",
      "release_date": "06/09/2021",
      "synopsis": "Uma poderosa narrativa visual sobre comunicação e isolamento, contada através da arte de Orlando Pedroso e Rui de Oliveira.",
      "availability": {
        "Online": ["Estante Virtual", "Amazon.com.br"]
      }
    },
    {
      "title": "Autorretrato",
      "author": "Renata Bueno",
      "imprint": "Perabook",
      "release_date": "21/02/2024",
      "synopsis": "Um livro-objeto que convida o leitor a explorar sua própria identidade através da arte. Criado e ilustrado por Renata Bueno.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Loja Elo Editorial"]
      }
    },
    {
      "title": "Barata Tonta",
      "author": "Flávia Savary",
      "imprint": "Elo Editora",
      "release_date": "14/06/2022",
      "synopsis": "Um poema divertido sobre uma barata que se perde e fica tonta. Com as ilustrações bem-humoradas de Bruno Nunes.",
      "availability": {
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "Barco a Vela",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Perabook",
      "release_date": "19/07/2023",
      "synopsis": "A sensação de liberdade e aventura de velejar em um poema visual. Com a arte serena de Paula Kranz, o livro é um convite à contemplação.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "Bate-boca",
      "author": "Claudio Fragata e Raquel Matsushita",
      "imprint": "Elo Editora",
      "release_date": "08/03/2022",
      "synopsis": "Um diálogo poético e visual sobre discussões e pontos de vista. Escrito e ilustrado pela dupla Claudio Fragata e Raquel Matsushita.",
      "availability": {
        "Online": ["Loja Elo Editorial", "Amazon.com.br"]
      }
    },
    {
      "title": "Baticum e o Tambor Real",
      "author": "Cristiane Velasco e Marina Siqueira",
      "imprint": "Perabook",
      "release_date": "23/05/2023",
      "synopsis": "Uma história sobre ritmo, ancestralidade e a cultura afro-brasileira. Com as vibrantes ilustrações de Bruna Lubambo.",
      "availability": {
        "Salvador": ["Livraria LDM"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "Bichão sem coração",
      "author": "Cristiane Velasco",
      "imprint": "Elo Editora",
      "release_date": "12/09/2023",
      "synopsis": "Será que o 'bichão' é tão mau assim? Uma história sobre aparências e a importância de conhecer o outro de verdade. Ilustrado por Bruna Lubambo.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "Bloom",
      "author": "Neville",
      "imprint": "Perabook",
      "release_date": "22/05/2023",
      "synopsis": "Uma graphic novel poética sobre crescimento pessoal e a beleza de florescer no seu próprio tempo. Com a arte marcante de Guilherme Asthma.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Blooks Livraria"],
        "Curitiba": ["Livrarias Curitiba"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    }
  ]
}
    """,
    agent=analista_catalogo_comercial,
)

In [8]:
find_stores_selling_book_task = Task(
    description=""" 
        Você vai receber {title} e, opcionalmente, {city}. 
        Procure os pontos de venda do livro. Se a cidade vier e existir no catálogo, use-a. 
        Se a cidade não vier ou não existir, aplique o fallback e retorne as opções de availability['Online'].
    """,
    expected_output="""
        Retorne APENAS um JSON enxuto (sem comentários) com:
        {
          "title"
          "city_used"
          "source" 
          "stores"    
        }
        'Se o livro não existir, retorne: {"not_found": true, "title": "{title}"}'
        
        
        Esse são os dados que nós temos dos livros:
    {
  "books": [
    {
      "title": "A Abelha",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Elo Editora",
      "release_date": "15/04/2022",
      "synopsis": "Uma obra delicada que explora o universo das abelhas e sua importância para a natureza. Com ilustrações de Olavo Costa, o livro é uma jornada poética e educativa.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Cultura"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A Baleia-azul",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "10/09/2023",
      "synopsis": "Acompanhe a vida majestosa do maior animal do planeta. Com ilustrações de Felipe Tognoli, esta obra mergulha nos mistérios dos oceanos de forma cativante.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br", "Magazine Luiza", "Submarino"]
      }
    },
    {
      "title": "A Bicicleta do Tempo",
      "author": "Daniela Pinotti e Marcelo Maluf",
      "imprint": "Elo Editora",
      "release_date": "20/05/2021",
      "synopsis": "Uma viagem fantástica sobre duas rodas que atravessa gerações e histórias. Ilustrado por Matheus Furtado, este livro fala sobre memória, amizade e as voltas que a vida dá.",
      "availability": {
        "Curitiba": ["Livrarias Curitiba"],
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Americanas.com"]
      }
    },
    {
      "title": "A Borboleta",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "12/02/2024",
      "synopsis": "Celebra a beleza e a transformação da vida através do ciclo da borboleta. As ilustrações de Rosana Ferreira dão cor e vida a esta delicada narrativa.",
      "availability": {
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A Cinderela dos Cajueiros",
      "author": "Severino Rodrigues",
      "imprint": "Elo Editora",
      "release_date": "30/07/2020",
      "synopsis": "Uma releitura do clássico conto de fadas, ambientado no sertão nordestino. Com a arte de Rodrigo Mafra, a história ganha novos sabores e cores.",
      "availability": {
        "Salvador": ["Livraria LDM"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A conquista de um sonho",
      "author": "Regina Drummond",
      "imprint": "Perabook",
      "release_date": "18/11/2022",
      "synopsis": "Uma história inspiradora sobre perseverança e a importância de lutar por seus objetivos. Ilustrado pela própria autora, Regina Drummond.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Leitura"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Cuca",
      "author": "Theo de Oliveira",
      "imprint": "Elo Editora",
      "release_date": "01/08/2021",
      "synopsis": "Mergulhe no folclore brasileiro com esta história sobre a Cuca, a temível feiticeira com cabeça de jacaré. Com vibrantes ilustrações de Danilo Taqueto.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Salvador": ["Livraria LDM"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "A Escola Lá Fora",
      "author": "Janaina Tokitaka",
      "imprint": "Perabook",
      "release_date": "25/03/2023",
      "synopsis": "Uma reflexão sobre o aprendizado que acontece fora das paredes da sala de aula. Escrito e ilustrado por Janaina Tokitaka, o livro convida a explorar o mundo.",
      "availability": {
        "São Paulo": ["Livraria Cultura", "Blooks Livraria"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A estranha atração dos planetas vizinhos",
      "author": "Claudio Fragata",
      "imprint": "Elo Editora",
      "release_date": "09/06/2024",
      "synopsis": "Uma história sobre conexões inesperadas e as forças que nos unem. Escrito e ilustrado com a sensibilidade de Claudio Fragata.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br (Pré-venda)"]
      }
    },
    {
      "title": "A Fabulosa Arca de Noá",
      "author": "Leandro Carloni",
      "imprint": "Perabook",
      "release_date": "14/10/2021",
      "synopsis": "Uma divertida e poética releitura da história da Arca de Noé, com personagens cativantes e ilustrações encantadoras de Fernanda Ozilak.",
      "availability": {
        "Curitiba": ["Livrarias Curitiba"],
        "Porto Alegre": ["Livraria Cameron"],
        "Online": ["Amazon.com.br", "Americanas.com"]
      }
    },
    {
      "title": "A Família Formiga",
      "author": "Augusto Pessoa",
      "imprint": "Elo Editora",
      "release_date": "22/08/2022",
      "synopsis": "Descubra o dia a dia de uma agitada família de formigas e aprenda sobre trabalho em equipe e união. Com as divertidas ilustrações de Heitor Neto.",
      "availability": {
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Fantástica Viagem de Okiké, o Macaquinho de Nariz Vermelho",
      "author": "G. Guillen e Marcel Tenório da",
      "imprint": "Perabook",
      "release_date": "03/03/2020",
      "synopsis": "A jornada de um macaquinho especial em busca de aceitação e de seu lugar no mundo. As ilustrações de Cris Mendes dão vida a essa aventura na floresta.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A Formiga",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Elo Editora",
      "release_date": "19/04/2022",
      "synopsis": "Um olhar poético sobre a vida da formiga, sua força e persistência. Com a arte delicada de Ana Laura Alvarenga, o livro encanta pela simplicidade.",
      "availability": {
        "Online": ["Loja Elo Editorial", "Amazon.com.br"]
      }
    },
    {
      "title": "A Girafa",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "11/07/2023",
      "synopsis": "Explore a savana e conheça mais sobre a elegante girafa. Um livro informativo e visualmente rico, com ilustrações de Tainan Rocha.",
      "availability": {
        "São Paulo": ["Livraria Cultura"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A girafa que comeu a lua",
      "author": "Francine Bittencourt",
      "imprint": "Elo Editora",
      "release_date": "08/12/2021",
      "synopsis": "Uma história lúdica sobre sonhos, imaginação e até onde a curiosidade pode nos levar. Com as ilustrações mágicas de Vanessa Prezoto.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "A História de Virgulino",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Perabook",
      "release_date": "21/06/2022",
      "synopsis": "Uma narrativa inspirada na figura de Lampião, adaptada para o público jovem. Com a arte em estilo de xilogravura de Adelino Francisco dos Santos Neto.",
      "availability": {
        "Salvador": ["Livraria LDM"],
        "Online": ["Estante Virtual", "Amazon.com.br"]
      }
    },
    {
      "title": "A ilha do tesouro",
      "author": "Robert Louis Stevenson",
      "imprint": "Elo Editora",
      "release_date": "15/01/2022",
      "synopsis": "A clássica aventura de piratas e mapas, em uma edição especial com a arte moderna de Weberson Santiago, que dá nova vida à jornada de Jim Hawkins.",
      "availability": {
        "São Paulo": ["Livraria Cultura", "Livraria Leitura"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Porto Alegre": ["Livraria Cameron"],
        "Online": ["Amazon.com.br", "Americanas.com", "Estante Virtual"]
      }
    },
    {
      "title": "A jornada de Roweõ",
      "author": "Angela Pappiani",
      "imprint": "Perabook",
      "release_date": "29/09/2020",
      "synopsis": "Baseado em uma história real, o livro narra a jornada de um jovem Xavante. Com as impressionantes ilustrações de Mauricio Negro, é uma obra sobre cultura e resistência.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A Lenda da Vitória-régia",
      "author": "Alice Bella",
      "imprint": "Elo Editora",
      "release_date": "16/03/2021",
      "synopsis": "O reconto de uma das mais belas lendas do folclore amazônico. A arte de Danilo Taqueto captura a magia e o mistério da história da índia Naiá.",
      "availability": {
        "Manaus": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Lenda do Guaraná",
      "author": "Theo de Oliveira",
      "imprint": "Perabook",
      "release_date": "16/03/2021",
      "synopsis": "Descubra a origem mítica do guaraná nesta adaptação de uma famosa lenda indígena. As ilustrações de Danilo Taqueto complementam a narrativa.",
      "availability": {
        "Manaus": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "A mágica história de um livro encontrado",
      "author": "Stella Maris Rezende",
      "imprint": "Elo Editora",
      "release_date": "11/11/2022",
      "synopsis": "Um livro que fala sobre o amor pelos livros e as aventuras que eles podem nos proporcionar. Com ilustrações de Sid Meireles, é uma ode à leitura.",
      "availability": {
        "Belo Horizonte": ["Livraria Leitura"],
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A megera domada",
      "author": "William Shakespeare",
      "imprint": "Perabook",
      "release_date": "25/02/2023",
      "synopsis": "Uma adaptação moderna e acessível da clássica comédia de Shakespeare, sobre os embates entre Catarina e Petrúquio. Com a arte arrojada de Rodrigo Mafra.",
      "availability": {
        "Online": ["Amazon.com.br", "Loja Elo Editorial", "Americanas.com"]
      }
    },
    {
      "title": "A menina da varanda",
      "author": "Leo Cunha",
      "imprint": "Elo Editora",
      "release_date": "10/03/2018",
      "synopsis": "Uma garota curiosa observa o mundo da sua varanda, criando histórias sobre as pessoas que passam. Uma obra sensível ilustrada por Rogério Coelho.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Leitura"],
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial", "Magazine Luiza"]
      }
    },
    {
      "title": "A Minhoca",
      "author": "Paulla Thompson",
      "imprint": "Perabook",
      "release_date": "07/07/2024",
      "synopsis": "De forma divertida, este livro mostra a importância das minhocas para o solo e para o meio ambiente. Com as bem-humoradas ilustrações de Monge.",
      "availability": {
        "Online": ["Amazon.com.br (Pré-venda)"]
      }
    },
    {
      "title": "A Minhoca com Dor de Barriga",
      "author": "Adriana Yazbek",
      "imprint": "Elo Editora",
      "release_date": "14/01/2020",
      "synopsis": "O que será que uma minhoca comeu para ficar com dor de barriga? Uma história divertida e cheia de imaginação, ilustrada por Isabella Guizalberti.",
      "availability": {
        "São Paulo": ["Livraria Cultura"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A Mula sem Cabeça",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Perabook",
      "release_date": "28/10/2021",
      "synopsis": "Mais uma lenda do folclore brasileiro ganha vida nesta obra. Descubra a história da Mula sem Cabeça com as ilustrações de Heitor Neto.",
      "availability": {
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "A Poça d’Água",
      "author": "Ana Rapha Nunes",
      "imprint": "Elo Editora",
      "release_date": "18/05/2023",
      "synopsis": "Uma simples poça d'água pode ser um universo de descobertas e brincadeiras. Com as criativas ilustrações de Paula Kranz.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Curitiba": ["Livrarias Curitiba"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A poesia do nome",
      "author": "Vários Autores",
      "imprint": "Perabook",
      "release_date": "05/11/2024",
      "synopsis": "Uma antologia que celebra o poder e a beleza dos nomes através de poemas e contos de diversos autores talentosos. Uma coletânea essencial.",
      "availability": {
        "Online": ["Loja Elo Editorial", "Amazon.com.br"]
      }
    },
    {
      "title": "A Princesa e o Jasmim",
      "author": "Georgina Martins",
      "imprint": "Elo Editora",
      "release_date": "12/04/2022",
      "synopsis": "Uma releitura poética de contos orientais, sobre uma princesa e uma flor de jasmim mágica. A arte de Ana Laura Alvarenga transporta o leitor para outro mundo.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "A Pulga",
      "author": "Alice Bella",
      "imprint": "Perabook",
      "release_date": "23/08/2023",
      "synopsis": "A vida de uma pulga aventureira que salta de um animal para outro em busca de emoção. Ilustrado de forma cômica por Rosana Ferreira.",
      "availability": {
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "A Tartaruga",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Elo Editora",
      "release_date": "15/04/2022",
      "synopsis": "Uma história sobre paciência, sabedoria e o ritmo da natureza, contada pela perspectiva de uma tartaruga. Com as ilustrações de Olavo Costa.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "A vaca que não parava de sonhar",
      "author": "Zil de Paula",
      "imprint": "Perabook",
      "release_date": "09/02/2021",
      "synopsis": "Conheça uma vaca que sonhava em voar, nadar e viajar pelo mundo. Uma história sobre imaginação e não ter medo de ser diferente, com ilustrações de Paula Kranz.",
      "availability": {
        "Belo Horizonte": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Americanas.com"]
      }
    },
    {
      "title": "A viagem",
      "author": "Jean Claude Alphen",
      "imprint": "Elo Editora",
      "release_date": "27/06/2023",
      "synopsis": "Um livro de imagens sem texto que narra a emocionante viagem de um menino e seu barco. Escrito e ilustrado por Jean-Claude Alphen.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Livraria Cultura"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "Abecê do macunaíma: O herói da nossa gente",
      "author": "Claudio Fragata",
      "imprint": "Perabook",
      "release_date": "01/02/2024",
      "synopsis": "Uma introdução divertida e acessível ao universo de Macunaíma, o herói sem nenhum caráter de Mário de Andrade. Com a arte icônica de Ciça Fittipaldi.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "Além do mar",
      "author": "Ana Rapha Nunes",
      "imprint": "Perabook",
      "release_date": "20/10/2025",
      "synopsis": "Uma delicada história sobre conexão e saudade. Ilustrado por Rafa Antón, é um livro sobre as viagens que fazemos por dentro e por fora.",
      "availability": {
        "Online": ["Amazon.com.br (Pré-venda)", "Loja Elo Editorial (Pré-venda)"]
      }
    },
    {
      "title": "Ana bola e outras histórias corajosas",
      "author": "Eliana Martins",
      "imprint": "Elo Editora",
      "release_date": "13/05/2021",
      "synopsis": "Uma coletânea de contos sobre meninas fortes e corajosas que enfrentam seus medos. Com as expressivas ilustrações de Tainan Rocha.",
      "availability": {
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "Ao sol do novo mundo",
      "author": "Edy Lima",
      "imprint": "Perabook",
      "release_date": "10/08/2022",
      "synopsis": "A saga de uma família de imigrantes que chega ao Brasil em busca de uma nova vida. Uma narrativa histórica com a arte de Taísa Borges.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Estante Virtual"]
      }
    },
    {
      "title": "As ceroulas do rei",
      "author": "Zil de Paula",
      "imprint": "Elo Editora",
      "release_date": "04/04/2023",
      "synopsis": "Uma releitura bem-humorada do conto 'A Roupa Nova do Rei', mostrando o que acontece quando a verdade é dita. Ilustrado por Marcelo Cardinal.",
      "availability": {
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "As Cores da Gatinha Mimi",
      "author": "Eliana Martins",
      "imprint": "Perabook",
      "release_date": "24/01/2022",
      "synopsis": "A gatinha Mimi muda de cor de acordo com suas emoções. Um livro encantador sobre sentimentos para os pequenos, com a arte de Cris Eich.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "As estranhas criaturas do Sr. K",
      "author": "Franz Kafka",
      "imprint": "Elo Editora",
      "release_date": "30/03/2024",
      "synopsis": "Uma adaptação de contos de Kafka para o público jovem, explorando o bizarro e o absurdo. Com a interpretação visual única de Nat Grego.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    },
    {
      "title": "As peripécias da senhora Adélia",
      "author": "Maria Amália Camargo",
      "imprint": "Perabook",
      "release_date": "17/10/2023",
      "synopsis": "Acompanhe as divertidas e inesperadas aventuras de uma senhora cheia de vida e energia. Ilustrado por Vanessa Prezoto.",
      "availability": {
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "As Torres",
      "author": "Orlando Pedroso",
      "imprint": "Elo Editora",
      "release_date": "06/09/2021",
      "synopsis": "Uma poderosa narrativa visual sobre comunicação e isolamento, contada através da arte de Orlando Pedroso e Rui de Oliveira.",
      "availability": {
        "Online": ["Estante Virtual", "Amazon.com.br"]
      }
    },
    {
      "title": "Autorretrato",
      "author": "Renata Bueno",
      "imprint": "Perabook",
      "release_date": "21/02/2024",
      "synopsis": "Um livro-objeto que convida o leitor a explorar sua própria identidade através da arte. Criado e ilustrado por Renata Bueno.",
      "availability": {
        "São Paulo": ["Blooks Livraria"],
        "Online": ["Loja Elo Editorial"]
      }
    },
    {
      "title": "Barata Tonta",
      "author": "Flávia Savary",
      "imprint": "Elo Editora",
      "release_date": "14/06/2022",
      "synopsis": "Um poema divertido sobre uma barata que se perde e fica tonta. Com as ilustrações bem-humoradas de Bruno Nunes.",
      "availability": {
        "Online": ["Amazon.com.br", "Magazine Luiza"]
      }
    },
    {
      "title": "Barco a Vela",
      "author": "Milton Célio de Oliveira Filho",
      "imprint": "Perabook",
      "release_date": "19/07/2023",
      "synopsis": "A sensação de liberdade e aventura de velejar em um poema visual. Com a arte serena de Paula Kranz, o livro é um convite à contemplação.",
      "availability": {
        "São Paulo": ["Livraria da Vila"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "Bate-boca",
      "author": "Claudio Fragata e Raquel Matsushita",
      "imprint": "Elo Editora",
      "release_date": "08/03/2022",
      "synopsis": "Um diálogo poético e visual sobre discussões e pontos de vista. Escrito e ilustrado pela dupla Claudio Fragata e Raquel Matsushita.",
      "availability": {
        "Online": ["Loja Elo Editorial", "Amazon.com.br"]
      }
    },
    {
      "title": "Baticum e o Tambor Real",
      "author": "Cristiane Velasco e Marina Siqueira",
      "imprint": "Perabook",
      "release_date": "23/05/2023",
      "synopsis": "Uma história sobre ritmo, ancestralidade e a cultura afro-brasileira. Com as vibrantes ilustrações de Bruna Lubambo.",
      "availability": {
        "Salvador": ["Livraria LDM"],
        "Rio de Janeiro": ["Livraria Travessa"],
        "Online": ["Amazon.com.br"]
      }
    },
    {
      "title": "Bichão sem coração",
      "author": "Cristiane Velasco",
      "imprint": "Elo Editora",
      "release_date": "12/09/2023",
      "synopsis": "Será que o 'bichão' é tão mau assim? Uma história sobre aparências e a importância de conhecer o outro de verdade. Ilustrado por Bruna Lubambo.",
      "availability": {
        "São Paulo": ["Livraria Leitura"],
        "Online": ["Amazon.com.br", "Submarino"]
      }
    },
    {
      "title": "Bloom",
      "author": "Neville",
      "imprint": "Perabook",
      "release_date": "22/05/2023",
      "synopsis": "Uma graphic novel poética sobre crescimento pessoal e a beleza de florescer no seu próprio tempo. Com a arte marcante de Guilherme Asthma.",
      "availability": {
        "São Paulo": ["Livraria da Vila", "Blooks Livraria"],
        "Curitiba": ["Livrarias Curitiba"],
        "Online": ["Amazon.com.br", "Loja Elo Editorial"]
      }
    }
  ]
}
     """,
    agent=analista_catalogo_comercial,
)

### Agente 3 - Suporte

In [9]:
suporte_autores = Agent(
    role="Agente de Suporte/Autores",
    goal=""" 
        Ajudar autores/leitores com dúvidas simples (ex.: Dúvida sobre submissão). 
        Quando o usuário quiser abrir um chamado, coletar os 4 campos obrigatórios 
        (name, email, subject, message). Se algo faltar, pergunte só o necessário, de forma gentil. 
        Com todos os campos em mãos, chamar a ferramenta de ticket e confirmar para o usuário 
        com ID e status retornados."
    """,
    backstory=""" 
        Você é um agente de suporte focado esclarecer de forma simples as dúvidas dos autores e leitores,
        além disso tem a função de abrir chamados de suporte quando necessário.
        Em dúvidas de submissão, responda com 2-4 linhas objetivas e ofereça abrir um ticket. 
        Ao abrir ticket, não prometa prazos: confirme ID e status 'open' e agradeça o contato.
    """,
    verbose=True,
    memory=True,
    allow_delegation=False,
    max_iterations=3
)

In [ ]:
open_support_ticket_task = Task(
    description=""" 
        Abra um ticket para o usuário. Você receberá os campos {name}, {email}, {subject}, {message}. 
        Se algum campo estiver ausente, apenas mostre para ele as respostas dos outros agentes, e pergunte se ele quer
        criar um ticket, solicitando o(s) campo(s) faltante(s).
        Quando tiver tudo, chame a ferramenta de ticket. 
        Depois, responda confirmando a abertura com o ID e o status retornados pela ferramenta.
    """,
    expected_output=""" 
        Retorne APENAS um JSON enxuto, se ele enviar todas as informações necessárias para ticket
        caso ele não envie todas as informações, retorne um texto apenas falando o nome do autor do livro 
        que foi feita a pesquisa, e algumas informações com a availability do livro, sem mais nada. ex:
        'O livro {title} do autor (aqui você deve colocar o autor do livro pesquisado) está disponível nas seguintes lojas: (aqui você vai passar a store da loja
        que o agente fez a pesquisa, se caso não tenha loja, apenas omita essa parte)'.

        Porém se ele enviar todas as informações necessárias para ticket, os campos {name}, {email}, {subject}, {message}
        se caso TODOS esse campos estejam preechidos corretamente retorne o JSON com a seguinte mensagem antes: 
        'Seu ticket foi aberto com sucesso, em breve uma agente especializado entrará em contato para mais detalhes!'
        {
          "id": "TCK-20250906-0007",
          "status": "open",
          "subject": "{subject}",
          "title": "(título do livro pesquisado)",
          "author": "(aqui vai o autor do livro pesquisado com base no título que ele passou)",
        }
    """,
    agent=suporte_autores
)

### Criação da Crew

In [18]:
crew = Crew(
    agents=[orquestrador, analista_catalogo_comercial, suporte_autores],
    tasks=[get_book_details_task, find_stores_selling_book_task, open_support_ticket_task],
    verbose=True,
    process= Process.sequential,
    embedder={"provider": "google", "config": {"model": "models/embedding-001"}},
    full_output=True,
    max_iter=15,
    manager_llm=llm
)

In [19]:
inputs = {
    "title": "A Abelha", 
    "city": "Sao Paulo",           
    "name": "Arthur Pedreca",           
    "email": "arthur@example.com",
    "subject": "Dúvida sobre o livro",
    "message": "Gostaria de saber mais sobre o livro 'A Abelha'."
}

result = crew.kickoff(inputs=inputs)
print(result.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: dd8508fe-359b-4845-aef5-d9fec8650331                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agente de Catálogo/Comercial                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Você vai receber um título de livro em A Abelha. Consulte o catálogo usando                            │
│          suas ferramentas oficiais e traga só o que realmente existe no dado.                                   │
│          Se encontrar, responda de forma curta e direta. Se não encontrar, diga                                 │
│          claramente que não está no catálogo.                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agente de Catálogo/Comercial                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  { "title": "A Abelha", "author": "Milton Célio de Oliveira Filho", "imprint": "Elo Editora", "release_date":   │
│  "15/04/2022", "synopsis": "Uma obra delicada que explora o universo das abelhas e sua importância para a       │
│  natureza. Com ilustrações de Olavo Costa, o livro é uma jornada poética e educativa." }                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1eb46401-7291-4b31-a939-05f1e0638944                                                                     │
│  Agent: Agente de Catálogo/Comercial                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agente de Suporte/Autores                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Seu ticket foi aberto com sucesso, em breve uma agente especialisto entrará em contato para mais detalhes!     │
│  {                                                                                                              │
│    "id": "TCK-20250906-0007",                                                                                   │
│    "status": "open",                                                                                            │
│    "subject": "Dúvida sobre o livro",                                                                           │
│    "title": "Arthur Pedreca",                                                                                   │
│    "author": "Milton Célio de Oliveira Filho"                                                                   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b9af62a9-2bb4-45fb-8198-f49a71c52e58                                                                     │
│  Agent: Agente de Suporte/Autores                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: dd8508fe-359b-4845-aef5-d9fec8650331                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Seu ticket foi aberto com sucesso, em breve uma agente especialisto entrará em contato para      │
│  mais detalhes!                                                                                                 │
│  {                                                                                                              │
│    "id": "TCK-20250906-0007",                                                                                   │
│    "status": "open",                                                                                            │
│    "subject": "Dúvida sobre o livro",                                                                           │
│    "title": "Arthur Pedreca",                                                                                   │
│    "author": "Milton Célio de Oliveira Filho"                                                                   │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Seu ticket foi aberto com sucesso, em breve uma agente especialisto entrará em contato para mais detalhes!  
{
  "id": "TCK-20250906-0007",
  "status": "open",
  "subject": "Dúvida sobre o livro",
  "title": "Arthur Pedreca",
  "author": "Milton Célio de Oliveira Filho"
}
